In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
#train.dropna(inplace=True)
train.drop_duplicates(inplace=True)
train.head()
label = train['Transported'].astype('bool')

train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.

PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.

HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.

CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.

Destination - The planet the passenger will be debarking to.

Age - The age of the passenger.

VIP - Whether the passenger has paid for special VIP service during the voyage.

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

Name - The first and last names of the passenger.

Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

In [ ]:
train.info()

In [ ]:
train.isna().sum(axis=0)

In [ ]:
train['HomePlanet']= train['HomePlanet'].fillna('Earth')
test['HomePlanet']= test['HomePlanet'].fillna('Earth')
train['Destination']= train['Destination'].fillna('TRAPPIST-1e')
test['Destination']= test['Destination'].fillna('TRAPPIST-1e')
train['CryoSleep']= train['CryoSleep'].fillna(False)
test['CryoSleep']= test['CryoSleep'].fillna(False)
train['VIP']= train['VIP'].fillna(False)
test['VIP']= test['VIP'].fillna(False)
train['Age']= train['Age'].fillna(train['Age'].mean())
test['Age']= test['Age'].fillna(train['Age'].mean())
train['Spa']= train['Spa'].fillna(train['Spa'].mean())
test['Spa']= test['Spa'].fillna(train['Spa'].mean())
train['FoodCourt']= train['FoodCourt'].fillna(train['FoodCourt'].mean())
test['FoodCourt']= test['FoodCourt'].fillna(train['FoodCourt'].mean())
train['ShoppingMall']= train['ShoppingMall'].fillna(train['ShoppingMall'].mean())
test['ShoppingMall']= test['ShoppingMall'].fillna(train['ShoppingMall'].mean())
train['RoomService']= train['RoomService'].fillna(train['RoomService'].mean())
test['RoomService']= test['RoomService'].fillna(test['RoomService'].mean())
train['VRDeck']= train['VRDeck'].fillna(train['VRDeck'].mean())
test['VRDeck']= test['VRDeck'].fillna(test['VRDeck'].mean())

In [ ]:
train.isna().sum(axis=0)

In [ ]:

train['PassengerGrp']= train['PassengerId'].str.split("_",expand=True)[0]
test['PassengerGrp']= test['PassengerId'].str.split("_",expand=True)[0]

In [ ]:
train['Group'] = train.groupby('PassengerGrp')['PassengerId'].transform('count')>1
test['Group'] = test.groupby('PassengerGrp')['PassengerId'].transform('count')>1
train.head()

In [ ]:
train[train["CryoSleep"].isna()]

In [ ]:
import seaborn as sns
sns.countplot(data=train, x="Transported", hue='VIP');

In [ ]:
train[['Deck','Num','Side']] = train.Cabin.str.split("/", expand=True)
test[['Deck','Num','Side']] = test.Cabin.str.split("/", expand=True)

train['Num'] =train['Num'].astype('float64')
test['Num'] =test['Num'].astype('float64')

In [ ]:
train.info()

In [ ]:
train['Deck']= train['Deck'].fillna('G')
train['Num']= train['Num'].fillna(train['Num'].mean())
train['Side']= train['Side'].fillna('S')

test['Deck']= test['Deck'].fillna('G')
test['Num']= test['Num'].fillna(train['Num'].mean())
test['Side']= test['Side'].fillna('S')

In [ ]:
train['VIP'] = train['VIP'].astype(int)
train['Group'] = train['Group'].astype(int)
train['Transported'] = train['Transported'].astype(int)
train['CryoSleep'] = train['CryoSleep'].astype(int)

test['VIP'] = test['VIP'].astype(int)
test['Group'] = test['Group'].astype(int)
test['CryoSleep'] = test['CryoSleep'].astype(int)


In [ ]:
train.drop(["Cabin","PassengerId","PassengerGrp","Name"], axis=1, inplace=True)
test.drop(["Cabin","PassengerId","PassengerGrp","Name"], axis=1, inplace=True)
cat_columns = train.select_dtypes(include='object').columns.values
num_columns = train.select_dtypes(exclude='object').columns.values
print(f""" 
Categorical columns:{cat_columns}

Numerical columns: {num_columns}
""")

In [ ]:
# Unique values; first 20
for col in cat_columns:
    print(f"""{col} :
------------------------------------
Unique_value:   {train[col].nunique()}
Null values: {train[col].isna().sum()}
Values:   
-------
{train[col].value_counts(dropna=False) if train[col].nunique()<20 else train[col].unique()[:20]}

    """)

In [ ]:
# Unique values; 
for col in num_columns:
    print(f"""{col} :
------------------------------------
Unique_value:   {train[col].nunique()}
Null values: {train[col].isna().sum()}
Range: {train[col].min()} - {train[col].max()} 
Values
-------
{train[col].value_counts(dropna=False) if train[col].nunique()<20 else train[col][:20].values}

    """)

In [ ]:
corr_matrix = train.corr()
from pandas.plotting import scatter_matrix

attributes = abs(corr_matrix['Transported'].sort_values(ascending=False))
print(attributes)


In [ ]:
num_columns = num_columns[num_columns!='Transported']

In [ ]:
churn_retain_stat = pd.DataFrame()
for col in cat_columns:
  # create crosstab for each attribute
  index = train[col]
  ct = pd.crosstab(index=index, columns=train['Transported'], normalize='index', colnames=[None]).reset_index()

  ct[col] = ct[col].apply(lambda x: f'{col.title()} ({x})')

  # rename the column
  ct.rename(columns={col:'attribute'}, inplace=True)

  # create a single dataframe
  churn_retain_stat = pd.concat([churn_retain_stat, ct])

churn_retain_stat.rename({0:'Transported',1:'NotTransported'}, axis=1,inplace=True)
churn_retain_stat.sort_values(by='Transported', ascending=False)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
num_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy="median")),
                         ('std,scaler', StandardScaler())
                         ])

In [ ]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
                                   ("num", num_pipeline, num_columns),
                                   ("cat", OneHotEncoder(), cat_columns)
])

In [ ]:
train_prepared = full_pipeline.fit_transform(train.drop('Transported', axis=1))
test_prepared = full_pipeline.transform(test)
train_prepared

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
tree_reg = RandomForestClassifier(n_estimators=100,random_state=1)
# Run cross validation
cv_results = cross_val_score(tree_reg, train_prepared, label, cv=5, scoring='roc_auc')

In [ ]:
print(cv_results)
print(np.mean(cv_results))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_prepared,label,test_size=0.33,random_state=42)


In [ ]:
import lightgbm as lgb
model_lgb = lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model_lgb.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],
          eval_metric='logloss')

In [ ]:
print('Training accuracy {:.4f}'.format(model_lgb.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model_lgb.score(x_test,y_test)))

In [ ]:
lgb.plot_importance(model_lgb)

In [ ]:
pred = model_lgb.predict(train_prepared)
len(pred),len(label)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(pred, label)
cm

In [ ]:
pred = model_lgb.predict(test_prepared)
pred

In [ ]:
len(pred), len(test_prepared)

In [ ]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.DataFrame({'PassengerId':test.PassengerId,'Transported':pred})
#submission['Transported']=submission['Transported'].apply(lambda x: x==1)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index = False)